In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras 
import requests
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

tf.__version__

'2.9.2'

In [ ]:
pathToTrainData='/content/drive/MyDrive/cs331FinalProject/Experiment/split2/train/'
pathToTestData = '/content/drive/MyDrive/cs331FinalProject/Experiment/split2/test/'

# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(pathToTrainData,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(pathToTestData,
                                            target_size = (224, 224))

Found 5081 images belonging to 20 classes.
Found 1260 images belonging to 20 classes.


In [ ]:
url = "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
r = requests.get(url, allow_redirects=True)
open('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', 'wb').write(r.content)

80134624

In [ ]:
# Part 2 - Building the CNN
input_tensor = Input(shape=(224, 224, 3))
vgg_weights = "/content/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = tf.keras.applications.VGG19(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = None,
)
base_model.load_weights(vgg_weights)
x = base_model.output
print(base_model.input)
print(base_model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#base_model.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()
#x = Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(20,activation="softmax")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
model = Sequential([ base_model, x, predictions ])
print(model)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 20)                10260     
                                                                 
Total params: 20,034,644
Trainable params: 20,034,644
Non-trainable params: 0
_________________________________________________________________


In [ ]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
def get_callbacks(name):
    return[
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_categorical_crossentropy',patience=1),
    ]

In [ ]:
base_learning_rate  = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="categorical_crossentropy",
              metrics=[tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])

In [ ]:
history = model.fit(x = training_set, epochs= 20, validation_data = test_set)

Epoch 1/20
159/159 [==============================] - 2791s 18s/step - loss: 2.8910 - categorical_crossentropy: 2.8908 - accuracy: 0.1216 - val_loss: 2.7437 - val_categorical_crossentropy: 2.7400 - val_accuracy: 0.1651
Epoch 2/20
159/159 [==============================] - 129s 809ms/step - loss: 2.6624 - categorical_crossentropy: 2.6624 - accuracy: 0.2305 - val_loss: 2.5443 - val_categorical_crossentropy: 2.5403 - val_accuracy: 0.3063
Epoch 3/20
159/159 [==============================] - 130s 818ms/step - loss: 2.4911 - categorical_crossentropy: 2.4910 - accuracy: 0.3352 - val_loss: 2.3841 - val_categorical_crossentropy: 2.3862 - val_accuracy: 0.3365
Epoch 4/20
159/159 [==============================] - 132s 832ms/step - loss: 2.3462 - categorical_crossentropy: 2.3459 - accuracy: 0.3936 - val_loss: 2.2541 - val_categorical_crossentropy: 2.2513 - val_accuracy: 0.4071
Epoch 5/20
159/159 [==============================] - 129s 814ms/step - loss: 2.2332 - categorical_crossentropy: 2.2333 -

In [ ]:
model.evaluate(test_set)

40/40 [==============================] - 15s 370ms/step - loss: 1.4764 - categorical_crossentropy: 1.4803 - accuracy: 0.6206


[1.4764453172683716, 1.480334758758545, 0.620634913444519]